<img src="https://i.imgur.com/6U6q5jQ.png"/>

In [ ]:
import geopandas as gpd

linkWorldMap='https://github.com/SocialAnalytics-StrategicIntelligence/GeoDFBasics_py/raw/main/maps/worldMap.gpkg'

countries=gpd.read_file(linkWorldMap, layer='countryBorders')
rivers=gpd.read_file(linkWorldMap, layer='riverLines')
cities=gpd.read_file(linkWorldMap, layer='cityPoints')


In [ ]:
peru=countries[countries.COUNTRY=='Peru']
citiesPeru_clipped = gpd.clip(gdf=cities,
                          mask=peru)
riversPeru_clipped = gpd.clip(gdf=rivers,
                               mask=peru)

In [ ]:
base = peru.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
citiesPeru_clipped.plot(marker='+', color='red', markersize=15,
                    ax=base)
riversPeru_clipped.plot(edgecolor='blue', linewidth=0.5,
                    ax=base)

In [ ]:
import os
peru.to_file(os.path.join("maps","peruMap.gpkg"),layer='border', driver="GPKG")
riversPeru_clipped.to_file(os.path.join("maps","peruMap.gpkg"),layer='rivers', driver="GPKG")
citiesPeru_clipped.to_file(os.path.join("maps","peruMap.gpkg"),layer='cities', driver="GPKG")

# ex3


In [ ]:
peru_5389=peru.to_crs(5389)

cities_peru_5389=citiesPeru_clipped.to_crs(peru_5389.crs)

rivers_peru_5389=riversPeru_clipped.to_crs(peru_5389.crs)

In [ ]:
base = peru_5389.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
cities_peru_5389.plot(marker='+', color='red', markersize=15,
                    ax=base)
rivers_peru_5389.plot(edgecolor='blue', linewidth=0.5,
                    ax=base)

In [ ]:
peru_5389.crs

In [ ]:
peru_5389.to_file(os.path.join("maps","peruMap_5389.gpkg"),layer='border', driver="GPKG")
rivers_peru_5389.to_file(os.path.join("maps","peruMap_5389.gpkg"),layer='rivers', driver="GPKG")
cities_peru_5389.to_file(os.path.join("maps","peruMap_5389.gpkg"),layer='cities', driver="GPKG")

In [ ]:
import pandas as pd 
infoairports=pd.read_csv(os.path.join("data","ourAirports.csv"))

# some rows

infoairports

In [ ]:
peruAirports=infoairports[infoairports.iso_country=='PE']
peruAirports

In [ ]:
airportsPeru=gpd.GeoDataFrame(data=peruAirports.copy(),
                 geometry=gpd.points_from_xy(peruAirports.longitude_deg,
                                             peruAirports.latitude_deg), 
                 crs=peru.crs.to_epsg())# the coordinates were in degrees - unprojected

In [ ]:
airportsPeru_5389=airportsPeru.to_crs(5389)

In [ ]:
## then

base = peru_5389.plot(color='white', edgecolor='black')
airportsPeru_5389.plot(ax=base)

In [ ]:
airportsPeru_5389.to_file(os.path.join("maps","peruMap_5389.gpkg"),layer='airports', driver="GPKG")

# EX4

In [ ]:
import pandas as pd
continentData=pd.read_csv(os.path.join('data','kagleInfoCountry.csv'))
continentData

In [ ]:
mapGitLink='https://github.com/SocialAnalytics-StrategicIntelligence/GeoDFBasics_py/raw/main/maps/theMapAndData.gpkg'

theMapAndData=gpd.read_file(mapGitLink, layer='fragility')

theMapAndData

In [ ]:
pd.unique(theMapAndData.iso3)

In [ ]:
set(pd.unique(theMapAndData.iso3))-set(continentData['alpha-3'])

In [ ]:
theMapAndData=theMapAndData.merge(continentData.loc[:,['region','sub-region','alpha-3']],left_on='iso3', right_on='alpha-3')

In [ ]:
colsNeeded=['COUNTRY','iso3','Total_ei5','Total_ei5_cat','geometry']
America_2023=theMapAndData[(theMapAndData.Year==2020) & (theMapAndData.region=='Americas')].loc[:, colsNeeded]
America_2023.info()

In [ ]:
America_2023_prjed=America_2023.to_crs(8858)
America_2023_prjed

In [ ]:
America_2023_prjed['centroids']=America_2023_prjed.centroid
America_2023_prjed

In [ ]:
# just the plot
base=America_2023_prjed.plot(facecolor='white',
                     edgecolor='lightgrey',
                     linewidth=0.3,
                     figsize=(8,8))

# another column, with the size of the POINTS (based on "Total_ei5")
America_2023_prjed["sizeOfMarker_ei5"]=[3**(Val+1) for Val in America_2023_prjed["Total_ei5"]]

# now, plotting the centroids (declared as the geometry)!!!
America_2023_prjed.set_geometry("centroids").plot(column="Total_ei5_cat",
                                         categorical=True,
                                         markersize=America_2023_prjed["sizeOfMarker_ei5"],
                                         cmap='Paired_r',
                                         legend=True,ax=base)

In [ ]:
colsNeeded=['COUNTRY','iso3','Total_ei5','Total_ei5_cat','geometry']
America_2023_prjed_polygons=America_2023_prjed.loc[:,colsNeeded]

colsNeeded=['COUNTRY','iso3','Total_ei5','Total_ei5_cat','centroids']
America_2023_prjed_centers=America_2023_prjed.loc[:,colsNeeded]
America_2023_prjed_centers.rename(columns={'centroids':'geometry'},inplace=True)
America_2023_prjed_centers=America_2023_prjed_centers.set_geometry('geometry')

In [ ]:
America_2023_prjed_centers.info()

In [ ]:
America_2023_prjed_polygons.to_file(os.path.join("maps","America_2023_prjed.gpkg"),layer='countries', driver="GPKG")
America_2023_prjed_centers.to_file(os.path.join("maps","America_2023_prjed.gpkg"),layer='centroids', driver="GPKG")